In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
base_geladinhos = pd.read_csv('geladinhos.csv')

In [4]:
# 1. Converter as colunas para o tipo datetime
base_geladinhos['data_cadastro'] = pd.to_datetime(base_geladinhos['data_cadastro'])
base_geladinhos['data_hora_pedido'] = pd.to_datetime(base_geladinhos['data_hora_pedido'])

# 2. Extrair componentes temporais
base_geladinhos['ano_cadastro'] = base_geladinhos['data_cadastro'].dt.year
base_geladinhos['mes_cadastro'] = base_geladinhos['data_cadastro'].dt.month
base_geladinhos['dia_cadastro'] = base_geladinhos['data_cadastro'].dt.day
base_geladinhos['dia_semana_cadastro'] = base_geladinhos['data_cadastro'].dt.dayofweek
base_geladinhos['dia_do_ano_cadastro'] = base_geladinhos['data_cadastro'].dt.dayofyear
base_geladinhos['semana_do_ano_cadastro'] = base_geladinhos['data_cadastro'].dt.isocalendar().week.astype(int)

base_geladinhos['ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.year
base_geladinhos['mes_pedido'] = base_geladinhos['data_hora_pedido'].dt.month
base_geladinhos['dia_pedido'] = base_geladinhos['data_hora_pedido'].dt.day
base_geladinhos['hora_pedido'] = base_geladinhos['data_hora_pedido'].dt.hour
base_geladinhos['minuto_pedido'] = base_geladinhos['data_hora_pedido'].dt.minute
base_geladinhos['dia_semana_pedido'] = base_geladinhos['data_hora_pedido'].dt.dayofweek
base_geladinhos['dia_do_ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.dayofyear
base_geladinhos['semana_do_ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.isocalendar().week.astype(int)

# 3. Criar features de tempo relativas (se desejar)
base_geladinhos['dias_desde_cadastro_ate_pedido'] = (base_geladinhos['data_hora_pedido'] - base_geladinhos['data_cadastro']).dt.days

def periodo_do_dia(hora):
    if 5 <= hora < 12:
        return 'manha'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noite'
base_geladinhos['periodo_dia_pedido'] = base_geladinhos['hora_pedido'].apply(periodo_do_dia)

# --- PASSO CRUCIAL: Remover as colunas originais de data/hora AGORA ---
base_geladinhos = base_geladinhos.drop(columns=['data_cadastro', 'data_hora_pedido'])

print("Tipos de dados após tratamento e remoção das colunas originais de data:")
print(base_geladinhos.dtypes)
print("\nColunas restantes:")
print(base_geladinhos.columns.tolist())

Tipos de dados após tratamento e remoção das colunas originais de data:
id_pedido                           int64
id_cliente                          int64
idade                               int64
genero                             object
cidade                             object
bairro                             object
é_estudante                          bool
frequencia_visita                  object
canal_compra                       object
forma_pagamento                    object
quantidade_itens                    int64
sabor                              object
categoria                          object
preco_unitario                    float64
valor_total_pedido                float64
clima                              object
ano_cadastro                        int32
mes_cadastro                        int32
dia_cadastro                        int32
dia_semana_cadastro                 int32
dia_do_ano_cadastro                 int32
semana_do_ano_cadastro              int64
ano_

In [5]:
# Separando features (X) e alvo (y)
X_geladinhos_df = base_geladinhos.drop('sabor', axis=1) # Ainda como DataFrame
y_geladinhos = base_geladinhos['sabor'].values # Seu alvo

# Colunas que serão convertidas para array NumPy
current_x_columns = X_geladinhos_df.columns.tolist()

# Convertendo X para array NumPy AGORA
X_geladinhos_np = X_geladinhos_df.values

print("\nVerificando os tipos de dados em X_geladinhos_df antes da conversão para NumPy:")
print(X_geladinhos_df.dtypes)
# Verifique se não há mais 'datetime64[ns]' ou 'object' que contenha Timestamps


Verificando os tipos de dados em X_geladinhos_df antes da conversão para NumPy:
id_pedido                           int64
id_cliente                          int64
idade                               int64
genero                             object
cidade                             object
bairro                             object
é_estudante                          bool
frequencia_visita                  object
canal_compra                       object
forma_pagamento                    object
quantidade_itens                    int64
categoria                          object
preco_unitario                    float64
valor_total_pedido                float64
clima                              object
ano_cadastro                        int32
mes_cadastro                        int32
dia_cadastro                        int32
dia_semana_cadastro                 int32
dia_do_ano_cadastro                 int32
semana_do_ano_cadastro              int64
ano_pedido                          i

In [6]:
from sklearn.preprocessing import LabelEncoder

# Instanciando os LabelEncoders para cada coluna categórica nominal
label_enconder_genero = LabelEncoder()
label_enconder_cidade = LabelEncoder()
label_enconder_bairro = LabelEncoder()
label_enconder_frequencia = LabelEncoder()
label_enconder_canal_compra = LabelEncoder()
label_enconder_forma_pagamento = LabelEncoder()
label_enconder_categorias = LabelEncoder()
label_encoder_clima = LabelEncoder()
label_encoder_periodo_dia = LabelEncoder() # Se 'periodo_dia_pedido' for uma nova coluna categórica

# Mapeando nomes das colunas para seus índices numéricos no array NumPy X_geladinhos_np
# Use 'current_x_columns' para encontrar os índices corretos
idx_genero = current_x_columns.index('genero')
idx_cidade = current_x_columns.index('cidade')
idx_bairro = current_x_columns.index('bairro')
idx_frequencia = current_x_columns.index('frequencia_visita')
idx_canal_compra = current_x_columns.index('canal_compra')
idx_forma_pagamento = current_x_columns.index('forma_pagamento')
idx_categoria = current_x_columns.index('categoria')
idx_clima = current_x_columns.index('clima')

# Se 'periodo_dia_pedido' foi criado e é categórico
if 'periodo_dia_pedido' in current_x_columns:
    idx_periodo_dia = current_x_columns.index('periodo_dia_pedido')
    X_geladinhos_np[:, idx_periodo_dia] = label_encoder_periodo_dia.fit_transform(X_geladinhos_np[:, idx_periodo_dia])

# Aplicando LabelEncoder às colunas do array NumPy X_geladinhos_np
X_geladinhos_np[:, idx_genero] = label_enconder_genero.fit_transform(X_geladinhos_np[:, idx_genero])
X_geladinhos_np[:, idx_cidade] = label_enconder_cidade.fit_transform(X_geladinhos_np[:, idx_cidade])
X_geladinhos_np[:, idx_bairro] = label_enconder_bairro.fit_transform(X_geladinhos_np[:, idx_bairro])
X_geladinhos_np[:, idx_frequencia] = label_enconder_frequencia.fit_transform(X_geladinhos_np[:, idx_frequencia])
X_geladinhos_np[:, idx_canal_compra] = label_enconder_canal_compra.fit_transform(X_geladinhos_np[:, idx_canal_compra])
X_geladinhos_np[:, idx_forma_pagamento] = label_enconder_forma_pagamento.fit_transform(X_geladinhos_np[:, idx_forma_pagamento])
X_geladinhos_np[:, idx_categoria] = label_enconder_categorias.fit_transform(X_geladinhos_np[:, idx_categoria])
X_geladinhos_np[:, idx_clima] = label_encoder_clima.fit_transform(X_geladinhos_np[:, idx_clima])

# Codificando a variável alvo 'sabor' (y_geladinhos)
label_encoder_sabor_target = LabelEncoder()
y_geladinhos_encoded = label_encoder_sabor_target.fit_transform(y_geladinhos)

print("\nTipos de dados das colunas categóricas em X_geladinhos_np (devem ser numéricos agora):")
# Uma forma de verificar é pegar uma amostra e ver os tipos, ou inspecionar o array
print(X_geladinhos_np[:5, [idx_genero, idx_cidade, idx_bairro]])


Tipos de dados das colunas categóricas em X_geladinhos_np (devem ser numéricos agora):
[[1 0 327]
 [0 0 46]
 [1 0 133]
 [1 0 442]
 [0 0 208]]


In [7]:
from sklearn.preprocessing import StandardScaler
import numpy as np # Importar numpy caso não tenha sido importado ainda


# Identifique os índices das colunas numéricas que devem ser escalonadas.
# Use 'current_x_columns' para ter certeza dos índices corretos.
numeric_cols_to_scale = [
    'idade',
    'quantidade_itens',
    'preco_unitario',
    'valor_total_pedido',
    'ano_cadastro',
    'mes_cadastro',
    'dia_cadastro',
    'dia_semana_cadastro',
    'dia_do_ano_cadastro',
    'semana_do_ano_cadastro',
    'ano_pedido',
    'mes_pedido',
    'dia_pedido',
    'hora_pedido',
    'minuto_pedido',
    'dia_semana_pedido',
    'dia_do_ano_pedido',
    'semana_do_ano_pedido',
    'dias_desde_cadastro_ate_pedido'
]

indices_to_scale = [current_x_columns.index(col) for col in numeric_cols_to_scale if col in current_x_columns]

scaler = StandardScaler()

# Aplica o scaler SOMENTE às colunas selecionadas
# Garanta que os dados nessas colunas são do tipo numérico (int, float) antes de escalonar
X_geladinhos_np[:, indices_to_scale] = X_geladinhos_np[:, indices_to_scale].astype(float) # Garante que são float

X_geladinhos_np[:, indices_to_scale] = scaler.fit_transform(X_geladinhos_np[:, indices_to_scale])

print("Média das colunas escalonadas (deve ser próximo de 0):")
print(np.mean(X_geladinhos_np[:, indices_to_scale], axis=0))
print("\nDesvio padrão das colunas escalonadas (deve ser próximo de 1):")


Média das colunas escalonadas (deve ser próximo de 0):
[1.4919177004912855e-16 -6.323309931222099e-17 0.0 -2.7308320535435194e-16
 5.130460500879508e-14 5.684341886080802e-18 -6.159517340620368e-17
 8.635314685534468e-17 -1.857403120197887e-17 -1.36590738719633e-16
 -6.871909707939494e-14 2.0194956817931597e-16 7.209233210403454e-17
 2.6756374893466272e-17 5.540012892879531e-17 7.660094780703731e-16
 -9.039435866498025e-17 -2.2093438190040616e-17 8.919531779838508e-17]

Desvio padrão das colunas escalonadas (deve ser próximo de 1):


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Dividindo os dados: 80% para treino, 20% para teste
X_train, X_test, y_train, y_test = train_test_split(
    X_geladinhos_np, y_geladinhos_encoded, test_size=0.2, random_state=42, stratify=y_geladinhos_encoded
)

print(f"Formato de X_train: {X_train.shape}")
print(f"Formato de X_test: {X_test.shape}")
print(f"Formato de y_train: {y_train.shape}")
print(f"Formato de y_test: {y_test.shape}")

# Instanciando e treinando o modelo Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

print("\nTreinando o modelo...")
model.fit(X_train, y_train)
print("Modelo treinado com sucesso!")

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)

# --- Avaliação do Modelo ---
print("\n--- Avaliação do Modelo ---")
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Modelo: {accuracy:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

NameError: name 'train_test_split' is not defined

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# --- ATENÇÃO: É CRUCIAL QUE OS OBJETOS ABAIXO TENHAM SIDO CRIADOS E 'FIT'ADOS NO SEU PIPELINE DE PRÉ-PROCESSAMENTO ANTERIOR ---
# Se você está rodando as células em sequência em um ambiente de notebook, eles já estarão disponíveis.
# Caso contrário, você precisaria carregar/re-fitá-los.

# Exemplo de como você teria esses objetos (apenas para ilustrar, eles já devem existir no seu ambiente)
# label_encoder_genero = LabelEncoder().fit(base_geladinhos['genero']) # Exemplo
# ... e assim por diante para todos os LabelEncoders, o scaler e o encoder do alvo.

# A maneira mais segura de garantir que você tem todos os objetos 'fit'ados é:
# 1. Rodar todo o seu pipeline de pré-processamento (desde o carregamento dos dados até o escalonamento)
#    em uma única sessão de kernel ou script.
# 2. Salvar/carregar esses objetos (encoders, scaler) usando `joblib` ou `pickle` se você precisar
#    usá-los em sessões diferentes. Por enquanto, vamos assumir que eles estão na memória.

# Relembrar os nomes das colunas de X_df na ordem correta
# Isso é essencial para saber onde cada dado de entrada deve ir.
# A variável `current_x_columns_after_processing` deve vir do seu código anterior.
# Se não estiver disponível, você pode recriá-la após o Label Encoding, mas antes do .values
# Exemplo:
# X_df = base_geladinhos.drop('sabor', axis=1) # Após todas as manipulações de data
# # Aplique Label Encoders aqui em X_df
# current_x_columns_after_processing = X_df.columns.tolist()


def sugerir_sabor_geladinho(dados_entrada_usuario: dict):
    """
    Sugere o sabor do geladinho com base nos dados fornecidos pelo usuário.

    Args:
        dados_entrada_usuario (dict): Um dicionário contendo os dados do usuário.
                                      As chaves do dicionário devem corresponder aos nomes
                                      das colunas do seu DataFrame original (antes de processar datas e LabelEncode).
                                      Exemplo:
                                      {
                                          'id_pedido': 99999,
                                          'id_cliente': 99999,
                                          'data_cadastro': '2023-01-15',
                                          'idade': 30,
                                          'genero': 'Feminino',
                                          'cidade': 'Santos',
                                          'bairro': 'Boqueirão',
                                          'é_estudante': True,
                                          'frequencia_visita': 'Semanal',
                                          'data_hora_pedido': '2024-06-15 10:30:00',
                                          'canal_compra': 'Online',
                                          'forma_pagamento': 'Cartão de Crédito',
                                          'quantidade_itens': 1,
                                          'preco_unitario': 7.0,
                                          'valor_total_pedido': 7.0,
                                          'clima': 'Quente'
                                      }

    Returns:
        str: O sabor de geladinho sugerido pelo modelo.
             Retorna "Erro no processamento dos dados." se houver um problema.
    """
    try:
        # 1. Criar um DataFrame a partir dos dados do usuário
        # Colunas devem ser as mesmas do seu base_geladinhos original, excluindo 'sabor'.
        # Isso é fundamental para manter a ordem e o nome das colunas durante o pré-processamento.
        input_df = pd.DataFrame([dados_entrada_usuario])

        # Garantir que a coluna 'é_estudante' é booleana
        input_df['é_estudante'] = input_df['é_estudante'].astype(bool)

        # 2. TRATAMENTO DAS COLUNAS DE DATA E HORA (EXATAMENTE COMO NO TREINO)
        input_df['data_cadastro'] = pd.to_datetime(input_df['data_cadastro'])
        input_df['data_hora_pedido'] = pd.to_datetime(input_df['data_hora_pedido'])

        input_df['ano_cadastro'] = input_df['data_cadastro'].dt.year
        input_df['mes_cadastro'] = input_df['data_cadastro'].dt.month
        input_df['dia_cadastro'] = input_df['data_cadastro'].dt.day
        input_df['dia_semana_cadastro'] = input_df['data_cadastro'].dt.dayofweek
        input_df['dia_do_ano_cadastro'] = input_df['data_cadastro'].dt.dayofyear
        input_df['semana_do_ano_cadastro'] = input_df['data_cadastro'].dt.isocalendar().week.astype(int)

        input_df['ano_pedido'] = input_df['data_hora_pedido'].dt.year
        input_df['mes_pedido'] = input_df['data_hora_pedido'].dt.month
        input_df['dia_pedido'] = input_df['data_hora_pedido'].dt.day
        input_df['hora_pedido'] = input_df['data_hora_pedido'].dt.hour
        input_df['minuto_pedido'] = input_df['data_hora_pedido'].dt.minute
        input_df['dia_semana_pedido'] = input_df['data_hora_pedido'].dt.dayofweek
        input_df['dia_do_ano_pedido'] = input_df['data_hora_pedido'].dt.dayofyear
        input_df['semana_do_ano_pedido'] = input_df['data_hora_pedido'].dt.isocalendar().week.astype(int)

        input_df['dias_desde_cadastro_ate_pedido'] = (input_df['data_hora_pedido'] - input_df['data_cadastro']).dt.days

        # Função período_do_dia deve ser a mesma
        def periodo_do_dia(hora):
            if 5 <= hora < 12:
                return 'manha'
            elif 12 <= hora < 18:
                return 'tarde'
            else:
                return 'noite'

        input_df['periodo_dia_pedido'] = input_df['hora_pedido'].apply(periodo_do_dia)

        # Remover as colunas originais de data/hora
        input_df = input_df.drop(columns=['data_cadastro', 'data_hora_pedido'])

        # Remover colunas que são IDs e que não foram usadas no treino do X
        # As colunas 'id_pedido', 'id_cliente' não devem ser passadas para o modelo
        # Remova-as se ainda estiverem no input_df
        input_df = input_df.drop(columns=['id_pedido', 'id_cliente'])

        # 3. APLICAR LABEL ENCODER NAS COLUNAS CATEGÓRICAS (EXATAMENTE COMO NO TREINO)
        # É CRUCIAL USAR OS MESMOS ENCODERS QUE FORAM FITADOS NO TREINO!
        # Use .transform() aqui, nunca .fit_transform() em novos dados.
        input_df['genero'] = label_encoder_genero.transform(input_df['genero'])
        input_df['cidade'] = label_encoder_cidade.transform(input_df['cidade'])
        input_df['bairro'] = label_encoder_bairro.transform(input_df['bairro'])
        input_df['frequencia_visita'] = label_encoder_frequencia.transform(input_df['frequencia_visita'])
        input_df['canal_compra'] = label_encoder_canal_compra.transform(input_df['canal_compra'])
        input_df['forma_pagamento'] = label_encoder_forma_pagamento.transform(input_df['forma_pagamento'])
        input_df['categoria'] = label_encoder_categorias.transform(input_df['categoria'])
        input_df['clima'] = label_encoder_clima.transform(input_df['clima'])
        input_df['periodo_dia_pedido'] = label_encoder_periodo_dia.transform(input_df['periodo_dia_pedido'])


        # Garantir que a coluna 'é_estudante' é int (se já não for do LabelEncoder)
        input_df['é_estudante'] = input_df['é_estudante'].astype(int)

        # 4. CONVERTER PARA ARRAY NUMPY E ESCALAR AS COLUNAS NUMÉRICAS
        # A ordem das colunas no input_np deve ser a MESMA ordem que X_train
        # Para isso, usaremos `current_x_columns_after_processing` para reordenar.

        # Garanta que todas as colunas de input_df são numéricas (int/float) ou já foram codificadas
        for col in input_df.columns:
            if input_df[col].dtype == 'object':
                # Isso deve pegar qualquer string que ainda não foi transformada
                # Exemplo: se uma nova categoria apareceu que o encoder não conhece, isso pode falhar.
                # Para LabelEncoder, a melhor prática é que as categorias de teste estejam nas categorias de treino.
                # Se houver uma categoria desconhecida, .transform() levantará um ValueError.
                # Você pode adicionar um bloco try-except aqui ou garantir que seus dados de teste
                # estejam dentro do vocabulário do encoder.
                pass # Se chegou até aqui, espera-se que não haja mais 'object' não numérico


        # Reordenar as colunas de input_df para corresponderem à ordem de X_train
        # `current_x_columns_after_processing` é a lista de colunas de X_df antes do .values no treino
        input_np = input_df[current_x_columns_after_processing].values

        # Escalar as colunas numéricas (EXATAMENTE COMO NO TREINO, USANDO O MESMO SCALER)
        # Lembre-se dos `indices_to_scale` que você calculou antes
        input_np[:, indices_to_scale] = scaler.transform(input_np[:, indices_to_scale])

        # 5. Fazer a previsão
        previsao_codificada = model.predict(input_np)

        # 6. Decodificar a previsão de volta para o nome do sabor
        sabor_sugerido = label_encoder_sabor_target.inverse_transform(previsao_codificada)

        return sabor_sugerido[0] # Retorna o primeiro (e único) elemento do array

    except ValueError as ve:
        # Isso pode acontecer se uma categoria de entrada não foi vista durante o treinamento (em .transform())
        return f"Erro: Uma categoria de entrada não foi reconhecida. Detalhes: {ve}"
    except Exception as e:
        return f"Ocorreu um erro no processamento dos dados ou na previsão: {e}"

In [10]:
# Exemplo de dados de entrada de um novo cliente
novos_dados_cliente = {
    'id_pedido': 10001,
    'id_cliente': 10001,
    'data_cadastro': '2023-05-10', # Formato 'YYYY-MM-DD'
    'idade': 28,
    'genero': 'Feminino',
    'cidade': 'Santos',
    'bairro': 'Ponta da Praia', # Assegure-se que este bairro foi visto no treino ou que o encoder lida com novos
    'é_estudante': False,
    'frequencia_visita': 'Mensal',
    'data_hora_pedido': '2024-06-15 15:45:00', # Formato 'YYYY-MM-DD HH:MM:SS'
    'canal_compra': 'Aplicativo',
    'forma_pagamento': 'PIX',
    'quantidade_itens': 3,
    'preco_unitario': 6.5,
    'valor_total_pedido': 19.5,
    'clima': 'Frio'
}

sabor_sugerido = sugerir_sabor_geladinho(novos_dados_cliente)
print(f"\nPara os dados fornecidos, o sabor de geladinho sugerido é: {sabor_sugerido}")

# Outro exemplo
novos_dados_cliente_2 = {
    'id_pedido': 10002,
    'id_cliente': 10002,
    'data_cadastro': '2024-01-01',
    'idade': 45,
    'genero': 'Masculino',
    'cidade': 'São Vicente',
    'bairro': 'Centro',
    'é_estudante': True,
    'frequencia_visita': 'Semanal',
    'data_hora_pedido': '2024-06-15 20:00:00',
    'canal_compra': 'Unidade Física Boqueirão',
    'forma_pagamento': 'Dinheiro',
    'quantidade_itens': 2,
    'preco_unitario': 7.0,
    'valor_total_pedido': 14.0,
    'clima': 'Quente'
}

sabor_sugerido_2 = sugerir_sabor_geladinho(novos_dados_cliente_2)
print(f"\nPara os dados do cliente 2, o sabor de geladinho sugerido é: {sabor_sugerido_2}")


Para os dados fornecidos, o sabor de geladinho sugerido é: Ocorreu um erro no processamento dos dados ou na previsão: name 'label_encoder_genero' is not defined

Para os dados do cliente 2, o sabor de geladinho sugerido é: Ocorreu um erro no processamento dos dados ou na previsão: name 'label_encoder_genero' is not defined


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- PARTE 1: PRÉ-PROCESSAMENTO DOS DADOS (Rodar tudo isso primeiro!) ---
base_geladinhos = pd.read_csv('geladinhos.csv')

print("Iniciando o pré-processamento dos dados...")

# 1. Tratamento das colunas de data e hora
base_geladinhos['data_cadastro'] = pd.to_datetime(base_geladinhos['data_cadastro'])
base_geladinhos['data_hora_pedido'] = pd.to_datetime(base_geladinhos['data_hora_pedido'])

base_geladinhos['ano_cadastro'] = base_geladinhos['data_cadastro'].dt.year
base_geladinhos['mes_cadastro'] = base_geladinhos['data_cadastro'].dt.month
base_geladinhos['dia_cadastro'] = base_geladinhos['data_cadastro'].dt.day
base_geladinhos['dia_semana_cadastro'] = base_geladinhos['data_cadastro'].dt.dayofweek
base_geladinhos['dia_do_ano_cadastro'] = base_geladinhos['data_cadastro'].dt.dayofyear
base_geladinhos['semana_do_ano_cadastro'] = base_geladinhos['data_cadastro'].dt.isocalendar().week.astype(int)

base_geladinhos['ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.year
base_geladinhos['mes_pedido'] = base_geladinhos['data_hora_pedido'].dt.month
base_geladinhos['dia_pedido'] = base_geladinhos['data_hora_pedido'].dt.day
base_geladinhos['hora_pedido'] = base_geladinhos['data_hora_pedido'].dt.hour
base_geladinhos['minuto_pedido'] = base_geladinhos['data_hora_pedido'].dt.minute
base_geladinhos['dia_semana_pedido'] = base_geladinhos['data_hora_pedido'].dt.dayofweek
base_geladinhos['dia_do_ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.dayofyear
base_geladinhos['semana_do_ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.isocalendar().week.astype(int)

base_geladinhos['dias_desde_cadastro_ate_pedido'] = (base_geladinhos['data_hora_pedido'] - base_geladinhos['data_cadastro']).dt.days

def periodo_do_dia(hora):
    if 5 <= hora < 12:
        return 'manha'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noite'

base_geladinhos['periodo_dia_pedido'] = base_geladinhos['hora_pedido'].apply(periodo_do_dia)

# REMOVER as colunas originais de data/hora AGORA
base_geladinhos = base_geladinhos.drop(columns=['data_cadastro', 'data_hora_pedido'])
print("  - Colunas de data/hora processadas e removidas.")

# 2. Dividir em Features (X_df) e Alvo (y_geladinhos)
# Remova também 'id_pedido' e 'id_cliente' de X_df aqui, pois não serão features para o modelo
X_df = base_geladinhos.drop(columns=['sabor', 'id_pedido', 'id_cliente'], axis=1) # CORREÇÃO AQUI
y_geladinhos = base_geladinhos['sabor'].values
print("  - Dados divididos em X (DataFrame) e y (NumPy array), com IDs removidos de X.")

# 3. Aplicar LabelEncoder nas colunas categóricas do X_df
label_encoder_genero = LabelEncoder()
label_encoder_cidade = LabelEncoder()
label_encoder_bairro = LabelEncoder()
label_encoder_frequencia = LabelEncoder()
label_encoder_canal_compra = LabelEncoder()
label_encoder_forma_pagamento = LabelEncoder()
label_encoder_categorias = LabelEncoder()
label_encoder_clima = LabelEncoder()
label_encoder_periodo_dia = LabelEncoder()

most_frequent_categories = {}

def fit_and_store_most_frequent(encoder, column_name, df_column):
    encoder.fit(df_column)
    most_frequent = df_column.mode()[0]
    most_frequent_categories[column_name] = most_frequent
    return encoder.transform(df_column)

X_df['genero'] = fit_and_store_most_frequent(label_encoder_genero, 'genero', X_df['genero'])
X_df['cidade'] = fit_and_store_most_frequent(label_encoder_cidade, 'cidade', X_df['cidade'])
X_df['bairro'] = fit_and_store_most_frequent(label_encoder_bairro, 'bairro', X_df['bairro'])
X_df['frequencia_visita'] = fit_and_store_most_frequent(label_encoder_frequencia, 'frequencia_visita', X_df['frequencia_visita'])
X_df['canal_compra'] = fit_and_store_most_frequent(label_encoder_canal_compra, 'canal_compra', X_df['canal_compra'])
X_df['forma_pagamento'] = fit_and_store_most_frequent(label_encoder_forma_pagamento, 'forma_pagamento', X_df['forma_pagamento'])
X_df['categoria'] = fit_and_store_most_frequent(label_encoder_categorias, 'categoria', X_df['categoria'])
X_df['clima'] = fit_and_store_most_frequent(label_encoder_clima, 'clima', X_df['clima'])
X_df['periodo_dia_pedido'] = fit_and_store_most_frequent(label_encoder_periodo_dia, 'periodo_dia_pedido', X_df['periodo_dia_pedido'])

X_df['é_estudante'] = X_df['é_estudante'].astype(int)
print("  - Colunas categóricas em X_df codificadas com LabelEncoder.")

label_encoder_sabor_target = LabelEncoder()
y_geladinhos_encoded = label_encoder_sabor_target.fit_transform(y_geladinhos)
print("  - Variável alvo (sabor) codificada.")

# 4. Obter a lista de colunas APÓS TODAS AS REMOÇÕES E TRANSFORMAÇÕES NO DATAFRAME X_df
# ESTA É A LISTA QUE DEVE SER USADA PARA MANTER A ORDEM!
current_x_columns_after_processing = X_df.columns.tolist() # CORREÇÃO AQUI
print(f"  - Ordem final das colunas em X: {current_x_columns_after_processing}")


numeric_cols_to_scale = [
    'idade', 'quantidade_itens', 'preco_unitario', 'valor_total_pedido',
    'ano_cadastro', 'mes_cadastro', 'dia_cadastro', 'dia_semana_cadastro', 'dia_do_ano_cadastro', 'semana_do_ano_cadastro',
    'ano_pedido', 'mes_pedido', 'dia_pedido', 'hora_pedido', 'minuto_pedido', 'dia_semana_pedido', 'dia_do_ano_pedido', 'semana_do_ano_pedido',
    'dias_desde_cadastro_ate_pedido'
]

indices_to_scale = [current_x_columns_after_processing.index(col) for col in numeric_cols_to_scale if col in current_x_columns_after_processing]

X_geladinhos_np = X_df.values

scaler = StandardScaler()
X_geladinhos_np[:, indices_to_scale] = scaler.fit_transform(X_geladinhos_np[:, indices_to_scale])
print("  - Colunas numéricas escalonadas.")
print(f"  - Tipo de dados final de X_geladinhos_np: {X_geladinhos_np.dtype}")

# 5. Divisão em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X_geladinhos_np, y_geladinhos_encoded, test_size=0.2, random_state=42, stratify=y_geladinhos_encoded
)
print("  - Dados divididos em conjuntos de treino e teste.")

# 6. Treinamento do Modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
print("  - Treinando o modelo...")
model.fit(X_train, y_train)
print("  - Modelo treinado com sucesso!")
print("\nPré-processamento e treinamento concluídos com sucesso!")

# --- PARTE 2: FUNÇÃO DE SUGESTÃO E TESTE (Rodar esta parte DEPOIS da Parte 1) ---

def sugerir_sabor_geladinho(dados_entrada_usuario: dict):
    try:
        input_df = pd.DataFrame([dados_entrada_usuario])

        input_df['é_estudante'] = input_df['é_estudante'].astype(bool)

        # Tratamento de datas
        input_df['data_cadastro'] = pd.to_datetime(input_df['data_cadastro'])
        input_df['data_hora_pedido'] = pd.to_datetime(input_df['data_hora_pedido'])

        input_df['ano_cadastro'] = input_df['data_cadastro'].dt.year
        input_df['mes_cadastro'] = input_df['data_cadastro'].dt.month
        input_df['dia_cadastro'] = input_df['data_cadastro'].dt.day
        input_df['dia_semana_cadastro'] = input_df['data_cadastro'].dt.dayofweek
        input_df['dia_do_ano_cadastro'] = input_df['data_cadastro'].dt.dayofyear
        input_df['semana_do_ano_cadastro'] = input_df['data_cadastro'].dt.isocalendar().week.astype(int)

        input_df['ano_pedido'] = input_df['data_hora_pedido'].dt.year
        input_df['mes_pedido'] = input_df['data_hora_pedido'].dt.month
        input_df['dia_pedido'] = input_df['data_hora_pedido'].dt.day
        input_df['hora_pedido'] = input_df['data_hora_pedido'].dt.hour
        input_df['minuto_pedido'] = input_df['data_hora_pedido'].dt.minute
        input_df['dia_semana_pedido'] = input_df['data_hora_pedido'].dt.dayofweek
        input_df['dia_do_ano_pedido'] = input_df['data_hora_pedido'].dt.dayofyear
        input_df['semana_do_ano_pedido'] = input_df['data_hora_pedido'].dt.isocalendar().week.astype(int)

        input_df['dias_desde_cadastro_ate_pedido'] = (input_df['data_hora_pedido'] - input_df['data_cadastro']).dt.days

        input_df['periodo_dia_pedido'] = input_df['hora_pedido'].apply(periodo_do_dia)

        # Remover as colunas originais de data/hora e IDs de input_df
        input_df = input_df.drop(columns=['data_cadastro', 'data_hora_pedido', 'id_pedido', 'id_cliente']) # JÁ REMOVE AQUI

        # Aplicar LabelEncoder (usando os encoders GLOBALMENTE DISPONÍVEIS)
        encoders_map = {
            'genero': label_encoder_genero,
            'cidade': label_encoder_cidade,
            'bairro': label_encoder_bairro,
            'frequencia_visita': label_encoder_frequencia,
            'canal_compra': label_encoder_canal_compra,
            'forma_pagamento': label_encoder_forma_pagamento,
            'categoria': label_encoder_categorias,
            'clima': label_encoder_clima,
            'periodo_dia_pedido': label_encoder_periodo_dia
        }

        for col_name, encoder in encoders_map.items():
            if col_name in input_df.columns: # Verifica se a coluna existe no input_df atual
                freq_cat = most_frequent_categories.get(col_name)
                unknown_mask = ~input_df[col_name].isin(encoder.classes_)
                if unknown_mask.any():
                    print(f"  Aviso: Categoria(s) desconhecida(s) em '{col_name}' no input. Substituindo por '{freq_cat}'.")
                    input_df.loc[unknown_mask, col_name] = freq_cat
                input_df[col_name] = encoder.transform(input_df[col_name])

        input_df['é_estudante'] = input_df['é_estudante'].astype(int)

        # Converter para NumPy array e garantir a ordem das colunas
        # 'current_x_columns_after_processing' agora já NÃO CONTÉM 'id_pedido', 'id_cliente', 'sabor'
        input_np = input_df[current_x_columns_after_processing].values # AGORA ISSO DEVE FUNCIONAR

        # Escalar as colunas numéricas
        input_np[:, indices_to_scale] = scaler.transform(input_np[:, indices_to_scale])

        # Fazer a previsão
        previsao_codificada = model.predict(input_np)

        # Decodificar a previsão
        sabor_sugerido = label_encoder_sabor_target.inverse_transform(previsao_codificada)

        return sabor_sugerido[0]

    except Exception as e:
        return f"Ocorreu um erro no processamento dos dados ou na previsão: {e}"

Iniciando o pré-processamento dos dados...
  - Colunas de data/hora processadas e removidas.
  - Dados divididos em X (DataFrame) e y (NumPy array), com IDs removidos de X.
  - Colunas categóricas em X_df codificadas com LabelEncoder.
  - Variável alvo (sabor) codificada.
  - Ordem final das colunas em X: ['idade', 'genero', 'cidade', 'bairro', 'é_estudante', 'frequencia_visita', 'canal_compra', 'forma_pagamento', 'quantidade_itens', 'categoria', 'preco_unitario', 'valor_total_pedido', 'clima', 'ano_cadastro', 'mes_cadastro', 'dia_cadastro', 'dia_semana_cadastro', 'dia_do_ano_cadastro', 'semana_do_ano_cadastro', 'ano_pedido', 'mes_pedido', 'dia_pedido', 'hora_pedido', 'minuto_pedido', 'dia_semana_pedido', 'dia_do_ano_pedido', 'semana_do_ano_pedido', 'dias_desde_cadastro_ate_pedido', 'periodo_dia_pedido']
  - Colunas numéricas escalonadas.
  - Tipo de dados final de X_geladinhos_np: float64
  - Dados divididos em conjuntos de treino e teste.
  - Treinando o modelo...
  - Modelo treinado

In [12]:
# --- TESTANDO A FUNÇÃO ---

# Exemplo de dados de entrada de um novo cliente
novos_dados_cliente = {
    'id_pedido': 10001,
    'id_cliente': 10001,
    'data_cadastro': '2023-05-10',
    'idade': 28,
    'genero': 'Feminino',
    'cidade': 'Santos',
    'bairro': 'Ponta da Praia',
    'é_estudante': False,
    'frequencia_visita': 'Mensal',
    'data_hora_pedido': '2024-06-15 15:45:00',
    'canal_compra': 'Aplicativo',
    'forma_pagamento': 'PIX',
    'quantidade_itens': 3,
    'preco_unitario': 6.5,
    'valor_total_pedido': 19.5,
    'clima': 'Frio'
}

sabor_sugerido = sugerir_sabor_geladinho(novos_dados_cliente)
print(f"\nPara os dados fornecidos, o sabor de geladinho sugerido é: {sabor_sugerido}")

# Outro exemplo
novos_dados_cliente_2 = {
    'id_pedido': 10002,
    'id_cliente': 10002,
    'data_cadastro': '2024-01-01',
    'idade': 45,
    'genero': 'Masculino',
    'cidade': 'São Vicente',
    'bairro': 'Centro',
    'é_estudante': True,
    'frequencia_visita': 'Semanal',
    'data_hora_pedido': '2024-06-15 20:00:00',
    'canal_compra': 'Unidade Física Boqueirão',
    'forma_pagamento': 'Dinheiro',
    'quantidade_itens': 2,
    'preco_unitario': 7.0,
    'valor_total_pedido': 14.0,
    'clima': 'Quente'
}

sabor_sugerido_2 = sugerir_sabor_geladinho(novos_dados_cliente_2)
print(f"\nPara os dados do cliente 2, o sabor de geladinho sugerido é: {sabor_sugerido_2}")

  Aviso: Categoria(s) desconhecida(s) em 'bairro' no input. Substituindo por 'Cardoso'.
  Aviso: Categoria(s) desconhecida(s) em 'canal_compra' no input. Substituindo por 'Unidade Física Pompéia'.

Para os dados fornecidos, o sabor de geladinho sugerido é: Ocorreu um erro no processamento dos dados ou na previsão: "['categoria'] not in index"
  Aviso: Categoria(s) desconhecida(s) em 'cidade' no input. Substituindo por 'Santos'.

Para os dados do cliente 2, o sabor de geladinho sugerido é: Ocorreu um erro no processamento dos dados ou na previsão: "['categoria'] not in index"


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- PART 1: DATA PREPROCESSING (Run all this first!) ---
base_geladinhos = pd.read_csv('geladinhos.csv')


print("Starting data preprocessing...")

# Assuming 'base_geladinhos' is already loaded.
# If not, uncomment and load your data:
# base_geladinhos = pd.read_csv('your_data_file.csv') # Replace with your file path

# 1. Date and Time Column Handling
base_geladinhos['data_cadastro'] = pd.to_datetime(base_geladinhos['data_cadastro'])
base_geladinhos['data_hora_pedido'] = pd.to_datetime(base_geladinhos['data_hora_pedido'])

base_geladinhos['ano_cadastro'] = base_geladinhos['data_cadastro'].dt.year
base_geladinhos['mes_cadastro'] = base_geladinhos['data_cadastro'].dt.month
base_geladinhos['dia_cadastro'] = base_geladinhos['data_cadastro'].dt.day
base_geladinhos['dia_semana_cadastro'] = base_geladinhos['data_cadastro'].dt.dayofweek
base_geladinhos['dia_do_ano_cadastro'] = base_geladinhos['data_cadastro'].dt.dayofyear
base_geladinhos['semana_do_ano_cadastro'] = base_geladinhos['data_cadastro'].dt.isocalendar().week.astype(int)

base_geladinhos['ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.year
base_geladinhos['mes_pedido'] = base_geladinhos['data_hora_pedido'].dt.month
base_geladinhos['dia_pedido'] = base_geladinhos['data_hora_pedido'].dt.day
base_geladinhos['hora_pedido'] = base_geladinhos['data_hora_pedido'].dt.hour
base_geladinhos['minuto_pedido'] = base_geladinhos['data_hora_pedido'].dt.minute
base_geladinhos['dia_semana_pedido'] = base_geladinhos['data_hora_pedido'].dt.dayofweek
base_geladinhos['dia_do_ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.dayofyear
base_geladinhos['semana_do_ano_pedido'] = base_geladinhos['data_hora_pedido'].dt.isocalendar().week.astype(int)

base_geladinhos['dias_desde_cadastro_ate_pedido'] = (base_geladinhos['data_hora_pedido'] - base_geladinhos['data_cadastro']).dt.days

def periodo_do_dia(hora):
    if 5 <= hora < 12:
        return 'manha'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noite'

base_geladinhos['periodo_dia_pedido'] = base_geladinhos['hora_pedido'].apply(periodo_do_dia)

# Remove original date/time columns NOW
base_geladinhos = base_geladinhos.drop(columns=['data_cadastro', 'data_hora_pedido'])
print("  - Date/time columns processed and removed.")

# 2. Split into Features (X_df) and Target (y_geladinhos)
# Ensure 'id_pedido' and 'id_cliente' are dropped from X_df, but NOT 'categoria'
X_df = base_geladinhos.drop(columns=['sabor', 'id_pedido', 'id_cliente'], axis=1) # Corrected here
y_geladinhos = base_geladinhos['sabor'].values
print("  - Data split into X (DataFrame) and y (NumPy array), with IDs removed from X.")

# 3. Apply LabelEncoder to categorical columns in X_df
label_encoder_genero = LabelEncoder()
label_encoder_cidade = LabelEncoder()
label_encoder_bairro = LabelEncoder()
label_encoder_frequencia = LabelEncoder()
label_encoder_canal_compra = LabelEncoder()
label_encoder_forma_pagamento = LabelEncoder()
label_encoder_categorias = LabelEncoder() # This encoder is for 'categoria'
label_encoder_clima = LabelEncoder()
label_encoder_periodo_dia = LabelEncoder()

most_frequent_categories = {}

def fit_and_store_most_frequent(encoder, column_name, df_column):
    encoder.fit(df_column)
    most_frequent = df_column.mode()[0]
    most_frequent_categories[column_name] = most_frequent
    return encoder.transform(df_column)

X_df['genero'] = fit_and_store_most_frequent(label_encoder_genero, 'genero', X_df['genero'])
X_df['cidade'] = fit_and_store_most_frequent(label_encoder_cidade, 'cidade', X_df['cidade'])
X_df['bairro'] = fit_and_store_most_frequent(label_encoder_bairro, 'bairro', X_df['bairro'])
X_df['frequencia_visita'] = fit_and_store_most_frequent(label_encoder_frequencia, 'frequencia_visita', X_df['frequencia_visita'])
X_df['canal_compra'] = fit_and_store_most_frequent(label_encoder_canal_compra, 'canal_compra', X_df['canal_compra'])
X_df['forma_pagamento'] = fit_and_store_most_frequent(label_encoder_forma_pagamento, 'forma_pagamento', X_df['forma_pagamento'])
X_df['categoria'] = fit_and_store_most_frequent(label_encoder_categorias, 'categoria', X_df['categoria']) # Ensure 'categoria' is processed
X_df['clima'] = fit_and_store_most_frequent(label_encoder_clima, 'clima', X_df['clima'])
X_df['periodo_dia_pedido'] = fit_and_store_most_frequent(label_encoder_periodo_dia, 'periodo_dia_pedido', X_df['periodo_dia_pedido'])

X_df['é_estudante'] = X_df['é_estudante'].astype(int)
print("  - Categorical columns in X_df encoded with LabelEncoder.")

label_encoder_sabor_target = LabelEncoder()
y_geladinhos_encoded = label_encoder_sabor_target.fit_transform(y_geladinhos)
print("  - Target variable (sabor) encoded.")

# 4. Get the final list of columns AFTER ALL REMOVALS AND TRANSFORMATIONS IN X_df
# This list will be used to maintain order in the prediction function
current_x_columns_after_processing = X_df.columns.tolist()
print(f"  - Final order of columns in X: {current_x_columns_after_processing}")


numeric_cols_to_scale = [
    'idade', 'quantidade_itens', 'preco_unitario', 'valor_total_pedido',
    'ano_cadastro', 'mes_cadastro', 'dia_cadastro', 'dia_semana_cadastro', 'dia_do_ano_cadastro', 'semana_do_ano_cadastro',
    'ano_pedido', 'mes_pedido', 'dia_pedido', 'hora_pedido', 'minuto_pedido', 'dia_semana_pedido', 'dia_do_ano_pedido', 'semana_do_ano_pedido',
    'dias_desde_cadastro_ate_pedido'
]

indices_to_scale = [current_x_columns_after_processing.index(col) for col in numeric_cols_to_scale if col in current_x_columns_after_processing]

X_geladinhos_np = X_df.values

scaler = StandardScaler()
X_geladinhos_np[:, indices_to_scale] = scaler.fit_transform(X_geladinhos_np[:, indices_to_scale])
print("  - Numerical columns scaled.")
print(f"  - Final data type of X_geladinhos_np: {X_geladinhos_np.dtype}")

# 5. Split into Training and Test Sets
X_train, X_test, y_train, y_test = train_test_split(
    X_geladinhos_np, y_geladinhos_encoded, test_size=0.2, random_state=42, stratify=y_geladinhos_encoded
)
print("  - Data split into training and test sets.")

# 6. Model Training
model = RandomForestClassifier(n_estimators=100, random_state=42)
print("  - Training the model...")
model.fit(X_train, y_train)
print("  - Model trained successfully!")
print("\nPreprocessing and training completed successfully!")


def sugerir_sabor_geladinho(dados_entrada_usuario: dict):
    try:
        input_df = pd.DataFrame([dados_entrada_usuario])

        input_df['é_estudante'] = input_df['é_estudante'].astype(bool)

        # Date handling
        input_df['data_cadastro'] = pd.to_datetime(input_df['data_cadastro'])
        input_df['data_hora_pedido'] = pd.to_datetime(input_df['data_hora_pedido'])

        input_df['ano_cadastro'] = input_df['data_cadastro'].dt.year
        input_df['mes_cadastro'] = input_df['data_cadastro'].dt.month
        input_df['dia_cadastro'] = input_df['data_cadastro'].dt.day
        input_df['dia_semana_cadastro'] = input_df['data_cadastro'].dt.dayofweek
        input_df['dia_do_ano_cadastro'] = input_df['data_cadastro'].dt.dayofyear
        input_df['semana_do_ano_cadastro'] = input_df['data_cadastro'].dt.isocalendar().week.astype(int)

        input_df['ano_pedido'] = input_df['data_hora_pedido'].dt.year
        input_df['mes_pedido'] = input_df['data_hora_pedido'].dt.month
        input_df['dia_pedido'] = input_df['data_hora_pedido'].dt.day
        input_df['hora_pedido'] = input_df['data_hora_pedido'].dt.hour
        input_df['minuto_pedido'] = input_df['data_hora_pedido'].dt.minute
        input_df['dia_semana_pedido'] = input_df['data_hora_pedido'].dt.dayofweek
        input_df['dia_do_ano_pedido'] = input_df['data_hora_pedido'].dt.dayofyear
        input_df['semana_do_ano_pedido'] = input_df['data_hora_pedido'].dt.isocalendar().week.astype(int)

        input_df['dias_desde_cadastro_ate_pedido'] = (input_df['data_hora_pedido'] - input_df['data_cadastro']).dt.days

        input_df['periodo_dia_pedido'] = input_df['hora_pedido'].apply(periodo_do_dia)

        # Remove original date/time columns and IDs from input_df
        # ENSURE 'categoria' IS NOT IN THIS DROP LIST
        input_df = input_df.drop(columns=['data_cadastro', 'data_hora_pedido', 'id_pedido', 'id_cliente']) # 'categoria' is not here

        # Apply LabelEncoder (using globally available encoders)
        encoders_map = {
            'genero': label_encoder_genero,
            'cidade': label_encoder_cidade,
            'bairro': label_encoder_bairro,
            'frequencia_visita': label_encoder_frequencia,
            'canal_compra': label_encoder_canal_compra,
            'forma_pagamento': label_encoder_forma_pagamento,
            'categoria': label_encoder_categorias, # 'categoria' is here for encoding
            'clima': label_encoder_clima,
            'periodo_dia_pedido': label_encoder_periodo_dia
        }

        for col_name, encoder in encoders_map.items():
            if col_name in input_df.columns: # Check if the column exists in the current input_df
                freq_cat = most_frequent_categories.get(col_name)
                unknown_mask = ~input_df[col_name].isin(encoder.classes_)
                if unknown_mask.any():
                    print(f"  Warning: Unknown category(ies) in '{col_name}' in input. Replacing with '{freq_cat}'.")
                    input_df.loc[unknown_mask, col_name] = freq_cat
                input_df[col_name] = encoder.transform(input_df[col_name])

        input_df['é_estudante'] = input_df['é_estudante'].astype(int)

        # Convert to NumPy array and ensure column order
        # 'current_x_columns_after_processing' now correctly excludes 'id_pedido', 'id_cliente', 'sabor'
        # And importantly, 'categoria' *is* expected to be in both input_df and current_x_columns_after_processing
        input_np = input_df[current_x_columns_after_processing].values # This line should now work

        # Scale numerical columns
        input_np[:, indices_to_scale] = scaler.transform(input_np[:, indices_to_scale])

        # Make prediction
        previsao_codificada = model.predict(input_np)

        # Decode prediction
        sabor_sugerido = label_encoder_sabor_target.inverse_transform(previsao_codificada)

        return sabor_sugerido[0]

    except Exception as e:
        return f"An error occurred during data processing or prediction: {e}"

# --- TESTING THE FUNCTION ---

# Example user input data
new_client_data = {
    'id_pedido': 10001,
    'id_cliente': 10001,
    'data_cadastro': '2023-05-10',
    'idade': 28,
    'genero': 'Feminino',
    'cidade': 'Santos',
    'bairro': 'Ponta da Praia', # This category will be replaced if unseen
    'é_estudante': False,
    'frequencia_visita': 'Mensal',
    'data_hora_pedido': '2024-06-15 15:45:00',
    'canal_compra': 'Aplicativo',
    'forma_pagamento': 'PIX',
    'quantidade_itens': 3,
    'preco_unitario': 6.5,
    'valor_total_pedido': 19.5,
    'clima': 'Frio',
    'categoria': 'Geladinhos Premium' # Ensure 'categoria' is always provided in user input
}

suggested_flavor = sugerir_sabor_geladinho(new_client_data)
print(f"\nFor the provided data, the suggested geladinho flavor is: {suggested_flavor}")

# Another example
new_client_data_2 = {
    'id_pedido': 10002,
    'id_cliente': 10002,
    'data_cadastro': '2024-01-01',
    'idade': 45,
    'genero': 'Masculino',
    'cidade': 'São Vicente', # This category will be replaced if unseen
    'bairro': 'Centro',
    'é_estudante': True,
    'frequencia_visita': 'Semanal',
    'data_hora_pedido': '2024-06-15 20:00:00',
    'canal_compra': 'Unidade Física Boqueirão',
    'forma_pagamento': 'Dinheiro',
    'quantidade_itens': 2,
    'preco_unitario': 7.0,
    'valor_total_pedido': 14.0,
    'clima': 'Quente',
    'categoria': 'Geladinhos Tradicionais' # Ensure 'categoria' is always provided
}

suggested_flavor_2 = sugerir_sabor_geladinho(new_client_data_2)
print(f"\nFor client 2's data, the suggested geladinho flavor is: {suggested_flavor_2}")

Starting data preprocessing...
  - Date/time columns processed and removed.
  - Data split into X (DataFrame) and y (NumPy array), with IDs removed from X.
  - Categorical columns in X_df encoded with LabelEncoder.
  - Target variable (sabor) encoded.
  - Final order of columns in X: ['idade', 'genero', 'cidade', 'bairro', 'é_estudante', 'frequencia_visita', 'canal_compra', 'forma_pagamento', 'quantidade_itens', 'categoria', 'preco_unitario', 'valor_total_pedido', 'clima', 'ano_cadastro', 'mes_cadastro', 'dia_cadastro', 'dia_semana_cadastro', 'dia_do_ano_cadastro', 'semana_do_ano_cadastro', 'ano_pedido', 'mes_pedido', 'dia_pedido', 'hora_pedido', 'minuto_pedido', 'dia_semana_pedido', 'dia_do_ano_pedido', 'semana_do_ano_pedido', 'dias_desde_cadastro_ate_pedido', 'periodo_dia_pedido']
  - Numerical columns scaled.
  - Final data type of X_geladinhos_np: float64
  - Data split into training and test sets.
  - Training the model...
  - Model trained successfully!

Preprocessing and trainin